In [43]:
import re

num_lanes = 3
num_goals = 2
num_freecells = 2

In [66]:
L = lambda la,lb,i: [f"L{i}",f"[L{i}|L{i}s]"][i in [la,lb]]

def L_(la,lb,i):
    if i == la:
        r = f"L{la}s"
    elif i == lb:
        r = f"[L{la},L{lb}|L{lb}s]"
    else:
        r = f"L{i}"
    return r

la=3
lb=1

res=f"""
f(G1, G2, F1, F2, {L(la,lb,1)}, {L(la,lb,2)}, {L(la,lb,3)}, [S_|S], P, P_) :- L{la} \= b, (L{lb} = b ; (L{lb} \= b, L{lb} =:= L{la} + 1)), \+ member([{L_(la,lb,1)},{L_(la,lb,2)},{L_(la,lb,3)}],P), step_str(L{la}, 'L{la}', 'L{lb}', S_), f(G1, G2, F1, F2, {L_(la,lb,1)}, {L_(la,lb,2)}, {L_(la,lb,3)}, S, [[{L_(la,lb,1)},{L_(la,lb,2)},{L_(la,lb,3)}]|P], P_).
""".strip()

In [69]:
def lanes_to_lanes(num_lanes):
    L = lambda la,lb,i: [f"L{i}",f"[L{i}|L{i}s]"][i in [la,lb]]
    
    def L_(la,lb,i):
        if i == la:
            r = f"L{la}s"
        elif i == lb:
            r = f"[L{la},L{lb}|L{lb}s]"
        else:
            r = f"L{i}"
        return r
    
    res = []
    for la in range(1,num_lanes+1):
        for lb in range(1,num_lanes+1):
            res0 = f"""
            f(G1, G2, 
                F1, F2, 
                {L(la,lb,1)}, {L(la,lb,2)}, {L(la,lb,3)}, 
                [S_|S], P, P_
            ) :- 
                L{la} \= b, (L{lb} = b ; (L{lb} \= b, L{lb} =:= L{la} + 1)), 
                \+ member([{L_(la,lb,1)},{L_(la,lb,2)},{L_(la,lb,3)}],P), 
                step_str(L{la}, 'L{la}', 'L{lb}', S_), 
                f(G1, G2, 
                    F1, F2, 
                    {L_(la,lb,1)}, {L_(la,lb,2)}, {L_(la,lb,3)}, 
                    S, [[{L_(la,lb,1)},{L_(la,lb,2)},{L_(la,lb,3)}]|P], P_
                ).
            """
            res.append(re.sub("\n            ","\n",res0).lstrip())

    return "\n".join(res)

In [70]:
lanes_to_lanes(num_lanes)

"f(G1, G2, \n    F1, F2, \n    [L1|L1s], L2, L3, \n    [S_|S], P, P_\n) :- \n    L1 \\= b, (L1 = b ; (L1 \\= b, L1 =:= L1 + 1)), \n    \\+ member([L1s,L2,L3],P), \n    step_str(L1, 'L1', 'L1', S_), \n    f(G1, G2, \n        F1, F2, \n        L1s, L2, L3, \n        S, [[L1s,L2,L3]|P], P_\n    ).\n\nf(G1, G2, \n    F1, F2, \n    [L1|L1s], [L2|L2s], L3, \n    [S_|S], P, P_\n) :- \n    L1 \\= b, (L2 = b ; (L2 \\= b, L2 =:= L1 + 1)), \n    \\+ member([L1s,[L1,L2|L2s],L3],P), \n    step_str(L1, 'L1', 'L2', S_), \n    f(G1, G2, \n        F1, F2, \n        L1s, [L1,L2|L2s], L3, \n        S, [[L1s,[L1,L2|L2s],L3]|P], P_\n    ).\n\nf(G1, G2, \n    F1, F2, \n    [L1|L1s], L2, [L3|L3s], \n    [S_|S], P, P_\n) :- \n    L1 \\= b, (L3 = b ; (L3 \\= b, L3 =:= L1 + 1)), \n    \\+ member([L1s,L2,[L1,L3|L3s]],P), \n    step_str(L1, 'L1', 'L3', S_), \n    f(G1, G2, \n        F1, F2, \n        L1s, L2, [L1,L3|L3s], \n        S, [[L1s,L2,[L1,L3|L3s]]|P], P_\n    ).\n\nf(G1, G2, \n    F1, F2, \n    [L1|L1s],